# Lab 3: Word Embeddings and Language Modelling

Adam Ek

In this lab we'll explore constructing *static* word embeddings (i.e. word2vec) and building language models. We'll also evaluate these systems on intermediate tasks, namely word similarity and identifying "good" and "bad" sentences.

* For this we'll use pytorch. Some basic operations that will be useful can be found here: https://jhui.github.io/2018/02/09/PyTorch-Basic-operations
* In general: we are not interested in getting state-of-the-art performance :) focus on the implementation and not results of your model. For this reason, you can use a subset of the dataset: the first 5000-10 000 sentences or so, on linux/mac: ```head -n 10000 inputfile > outputfile```. 
* If possible, use the MLTGpu, it will make everything faster :)

In [68]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import os

# for gpu, replace "cpu" with "cuda:n" where n is the index of the GPU
#device = torch.device('cpu')
device = torch.device('cuda:0')

# Word2Vec embeddings

In this first part we'll construct a word2vec model which will give us *static* word embeddings (that is, they are fixed after training).

After we've trained our model we will evaluate the embeddings obtained on a word similarity task.

## Formatting data


First we need to load some data, you can download the file on canvas under files/03-lab-data/wiki-corpus.txt. The file contains 50 000 sentences randomly selected from the complete wikipedia. Each line in the file contains one sentence. The sentences are whitespace tokenized.

Your first task is to create a dataset suitable for word2vec. That is, we define some ```window_size``` then iterate over all sentences in the dataset, putting the center word in one field and the context words in another (separate the fields with ```tab```).

For example, the sentece "this is a lab" with ```window size = 4``` will be formatted as:
```
center, context
---------------------
this    is a
is      this a lab
a       this is lab
lab     is a
```

this will be our training examples when training the word2vec model.

[3 marks]

In [1]:
import os

# Imports
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset

import time
from collections import namedtuple
import collections
import toolz

In [2]:
os.listdir('.')

['Word Embeddings and Language Models.ipynb',
 '.ipynb_checkpoints',
 'wiki-corpus.50000.txt',
 'testing.txt',
 'tutorial',
 'testing.tsv',
 'a_file.txt']

In [8]:
# read data
def corpus_reader(data_path):
    with open(data_path) as f:
        data = f.read().splitlines()

    return data

In [96]:
# Implement custom torch Dataset
class ContextDataset(Dataset):
    
    def __init__(self,
                 raw_texts,
                 window_size=4,
                 tokenizer=None,
                 lower=True,
                 min_freq=0,
                 unk_label='<unk>',
                 pad_label='<pad>',
                 verbose=False):
        
        """
        raw_texts: list of texts
        """
        
        self.verbose=verbose
        
        self.window_size= 4
        self.min_freq = min_freq
        self.unk_label, self.unk_idx = unk_label, 0
        self.pad_label, self.pad_idx = pad_label, 1
        
        
        if tokenizer is None:
            self.tokenizer = lambda x: x.replace('.',' ').replace(',', ' ').replace('"', ' ').split() #simple punctuation and whitespace
        self.lower = lower
        
        # Load Data
        #self.center, self.context = self.__load_tsv(self.tsv_path)
        self.center, self.context = self.create_dataset(raw_texts)
        
        # Vocabulary
        self.word_to_idx = dict()
        self.word_to_idx[self.unk_label] = self.unk_idx
        self.word_to_idx[self.pad_label] = self.pad_idx
        self.word_to_idx.update({word:idx+max(self.word_to_idx.values())+1 for idx, word in enumerate(np.unique(self.center))})

        # Word Counts
        # center represents every instance of every word in the text
        self.word_counts = pd.value_counts(self.center).to_dict()
        
        # Apply min_freq
        if min_freq:
            word_counts_over_min_freq = {k:v for k,v in self.word_counts.items() if v >= min_freq}
            self.word_to_idx = {k:v for k,v in self.word_to_idx.items() if ((k in word_counts_over_min_freq) or k in [unk_label, pad_label])}
            
        self.idx_to_word = {v:k for k,v in self.word_to_idx.items()}
        
        # Numericalize
        # Note I will do this in data_loader.
        #self.center = np.array([self.numericalize(w) for w in self.center])
        #self.context = np.array([np.array([self.numericalize(w) for w in sent]) for sent in self.context], dtype='object')
        
    
    def get_window_context(self, position_index, sentence_array, window_size):
        """
        returns: center_word, context_array

        sentence_array = ['this', 'is', 'a', 'lab', 'filler', 'filler']
        window_size = 4
        position_index = 1

        reutrns: is, [this, a, lab]
        """
        window_range = int(np.ceil(window_size/2)) #Assume window_size is even otherwise round up. (3 -> 4)
        context_indices = [int(i) for i in range(position_index-window_range, position_index+window_range+1) if (i>=0 and i!=position_index and i<len(sentence_array))] 
        context_array = [sentence_array[i] for i in context_indices]

        return sentence_array[position_index], context_array
    
    def create_dataset(self, raw_texts):
        
        t1 = time.time()
        c = 0
        contexts_dfs = []
        for sent_id, text in enumerate(raw_texts):
            
            if self.verbose:
                c+=1
                if c%1000 == 0:
                    t2 = time.time()
                    print(f'{c}/{len(raw_texts)}, dt = {round(t2-t1,3)}')
                    t1 = time.time()
                
            text = self.tokenize(text)
            
            windowed_contexts = pd.DataFrame([self.get_window_context(i, text, self.window_size) for i,_ in enumerate(text)], columns = ['center', 'context'])
            windowed_contexts['sentence_id'] = sent_id
            contexts_dfs.append(windowed_contexts) 
        
        dataset = pd.concat(contexts_dfs, axis=0, ignore_index=True)
        
        return np.array(dataset['center']), np.array(dataset['context'])
    
    def tokenize(self, string):
        if self.lower:
            string = string.lower()
        return np.array(self.tokenizer(string))

    def idx2word(self, idx_or_list):
        try:
            len(idx_or_list)
            return [self.idx_to_word.get(idx, self.unk_label) for idx in idx_or_list]
        except:
            return self.idx_to_word.get(idx_or_list, self.unk_label)
    
    def word2idx(self, word_or_list):
        
        if isinstance(word_or_list, str):
            return self.word_to_idx.get(word_or_list, self.unk_idx)
        else:
            return [self.word_to_idx.get(w, self.unk_idx) for w in word_or_list]
        
    def get_shuffled_data(self, seed=None): 
        np.random.seed(seed)
        
        rng = np.arange(len(self))
        np.random.shuffle(rng)
        return self.center[rng], self.context[rng]
        
    def __getitem__(self, idx):
        return self.center[idx], self.context[idx]
    
    def __len__(self):
        return len(self.center)
    


In [31]:
print('TEST data parser')
test_dataset = ContextDataset(['A bomb was thrown by an unknown party', 'coherence theory hypothesizes that a limited'], window_size=4)
print('----')
print('get_window_context test:', test_dataset.get_window_context(1, ['this', 'is', 'a', 'lab', 'filler', 'filler'], 4))
        

print('center:context')
for i,j in zip(test_dataset.center[0:5], test_dataset.context[0:5]):
    print(i,':', j)

print(test_dataset.word_to_idx)

TEST data parser
----
get_window_context test: ('is', ['this', 'a', 'lab'])
center:context
a : ['bomb', 'was']
bomb : ['a', 'was', 'thrown']
was : ['a', 'bomb', 'thrown', 'by']
thrown : ['bomb', 'was', 'by', 'an']
by : ['was', 'thrown', 'an', 'unknown']
{'<unk>': 0, '<pad>': 1, 'a': 2, 'an': 3, 'bomb': 4, 'by': 5, 'coherence': 6, 'hypothesizes': 7, 'limited': 8, 'party': 9, 'that': 10, 'theory': 11, 'thrown': 12, 'unknown': 13, 'was': 14}


In [130]:
# Actual Data

data_path = '03-lab-data/wiki-corpus.50000.txt'
raw_texts = corpus_reader(data_path)
dataset = ContextDataset(raw_texts, window_size=4, verbose=True)


1000/50000, dt = 1.666
2000/50000, dt = 1.695
3000/50000, dt = 1.66
4000/50000, dt = 1.655
5000/50000, dt = 1.663
6000/50000, dt = 1.643
7000/50000, dt = 1.67
8000/50000, dt = 1.691
9000/50000, dt = 1.689
10000/50000, dt = 1.65
11000/50000, dt = 1.633
12000/50000, dt = 1.492
13000/50000, dt = 1.607
14000/50000, dt = 1.61
15000/50000, dt = 4.07
16000/50000, dt = 1.406
17000/50000, dt = 1.412
18000/50000, dt = 1.41
19000/50000, dt = 1.398
20000/50000, dt = 1.398
21000/50000, dt = 1.385
22000/50000, dt = 1.401
23000/50000, dt = 1.399
24000/50000, dt = 1.382
25000/50000, dt = 1.413
26000/50000, dt = 1.382
27000/50000, dt = 1.372
28000/50000, dt = 1.352
29000/50000, dt = 1.346
30000/50000, dt = 1.347
31000/50000, dt = 1.368
32000/50000, dt = 1.36
33000/50000, dt = 1.341
34000/50000, dt = 1.355
35000/50000, dt = 1.364
36000/50000, dt = 1.449
37000/50000, dt = 1.405
38000/50000, dt = 1.406
39000/50000, dt = 1.406
40000/50000, dt = 1.419
41000/50000, dt = 1.418
42000/50000, dt = 1.392
43000/50

In [27]:
for i,j in zip(context_dataset.center[0:10], context_dataset.context[0:10]):
    print(i,':', j)

anarchist : ['historian', 'george']
historian : ['anarchist', 'george', 'woodcock']
george : ['anarchist', 'historian', 'woodcock', 'reports']
woodcock : ['historian', 'george', 'reports', 'that']
reports : ['george', 'woodcock', 'that', 'the']
that : ['woodcock', 'reports', 'the', 'annual']
the : ['reports', 'that', 'annual', 'congress']
annual : ['that', 'the', 'congress', 'of']
congress : ['the', 'annual', 'of', 'the']
of : ['annual', 'congress', 'the', 'international']


We sampled 50 000 senteces completely random from the *whole* wikipedia for our training data. Give some reasons why this is good, and why it might be bad. (*note*: We'll have a few questions like these, one or two reasons for and against is sufficient)

[2 marks]

**Notes**

torch tensor does not handle arrays of arrays of different length. So we need padding. Considering 


**GOOD**
- It is fair too assume wikipedia doesnt contain too much bias.

**BAD**
- 50k sentences spread out over all of wikipedia might too small a sample, as the contexts can vary a lot.

### Loading the data

We now need to load the data in an appropriate format for torchtext (https://torchtext.readthedocs.io/en/latest/). We'll use PyText for this and it'll follow the same structure as I showed you in the lecture (remember to lower-case all tokens). Create a function which returns a (bucket)iterator of the training data, and the vocabulary object (```Field```). 

(*hint1*: you can format the data such that the center word always is first, then you only need to use one field)

(*hint2*: the code I showed you during the leture is available in /files/pytorch_tutorial/ on canvas)

[4 marks]

In [21]:
print('TEST Dataset')
print('-------')

print('Test min_freq=2')
dataset = ContextDataSet('testing.tsv', min_freq=2)
print('test getitem:',dataset[0])
print('test len:', len(dataset))
print('vocab', dataset.word_to_idx)
print('count:\n', dataset.word_counts)
print()
print('Test min_freq=0')
dataset = ContextDataSet('testing.tsv', min_freq=0)
print('test getitem:',dataset[0])
print('test len:', len(dataset))
print('vocab', dataset.word_to_idx)

print()
print('test shuffle_data')
print('before:', dataset.center[0:2])

c,_ = dataset.get_shuffled_data(seed=123)
print('after:', c[0:2])

print('test word2idx context')
print(dataset.context[1])
print(dataset.word2idx(dataset.context[1]))

TEST Dataset
-------
Test min_freq=2
test getitem: ('a', array(['bomb', 'was'], dtype='<U4'))
test len: 14
vocab {'<unk>': 0, '<pad>': 1, 'a': 2}
count:
 {'a': 2, 'bomb': 1, 'was': 1, 'thrown': 1, 'by': 1, 'an': 1, 'unknown': 1, 'party': 1, 'coherence': 1, 'theory': 1, 'hypothesizes': 1, 'that': 1, 'limited': 1}

Test min_freq=0
test getitem: ('a', array(['bomb', 'was'], dtype='<U4'))
test len: 14
vocab {'<unk>': 0, '<pad>': 1, 'a': 2, 'an': 3, 'bomb': 4, 'by': 5, 'coherence': 6, 'hypothesizes': 7, 'limited': 8, 'party': 9, 'that': 10, 'theory': 11, 'thrown': 12, 'unknown': 13, 'was': 14}

test shuffle_data
before: ['a' 'bomb']
after: ['party' 'hypothesizes']
test word2idx context
['a' 'was' 'thrown']
[2, 14, 12]


In [36]:
# Custom loader

from collections import namedtuple
from toolz import take
from itertools import tee

def get_loader(dataset, batch_size, shuffle=True):
    
    if shuffle:
        centers, contexts = dataset.get_shuffled_data() 
    else:
        centers = dataset.center
        contexts = dataset.context
        
    centers = iter(centers)
    contexts = iter(contexts)
    Batch = namedtuple('Batch', ['contexts', 'centers'])
    for i in range(int(len(dataset)/batch_size)+1):
        batch_centers = take(batch_size, centers)
        if not batch_centers:
            continue
        batch_contexts, bc2 = tee(take(batch_size, contexts))
        
        #padding and word2idx - this part i think is refered to as Collate in torch.
        batch_centers = torch.tensor(dataset.word2idx(batch_centers))
        max_len = max([len(x) for x in bc2])
        batch_contexts = torch.tensor([dataset.word2idx(s)+[dataset.pad_idx]*(max_len-len(s)) for s in batch_contexts])

        yield Batch(batch_contexts, batch_centers)
        
# Todo try with torch.DataLoader


In [37]:
print('TEST (CUSTOM) LOADER')
test_loader = get_loader(test_dataset, 5, shuffle=False)
for cont, cent in test_loader:
    print(np.shape(cont)) # Note: batch_first
    print(cent.tolist())
    print(test_dataset.idx2word(cent.tolist()))
    print([test_dataset.idx2word(c) for c in cont.tolist()])
    print()

TEST (CUSTOM) LOADER
torch.Size([5, 4])
[2, 4, 14, 12, 5]
['a', 'bomb', 'was', 'thrown', 'by']
[['bomb', 'was', '<pad>', '<pad>'], ['a', 'was', 'thrown', '<pad>'], ['a', 'bomb', 'thrown', 'by'], ['bomb', 'was', 'by', 'an'], ['was', 'thrown', 'an', 'unknown']]

torch.Size([5, 4])
[3, 13, 9, 6, 11]
['an', 'unknown', 'party', 'coherence', 'theory']
[['thrown', 'by', 'unknown', 'party'], ['by', 'an', 'party', '<pad>'], ['an', 'unknown', '<pad>', '<pad>'], ['theory', 'hypothesizes', '<pad>', '<pad>'], ['coherence', 'hypothesizes', 'that', '<pad>']]

torch.Size([4, 4])
[7, 10, 2, 8]
['hypothesizes', 'that', 'a', 'limited']
[['coherence', 'theory', 'that', 'a'], ['theory', 'hypothesizes', 'a', 'limited'], ['hypothesizes', 'that', 'limited', '<pad>'], ['that', 'a', '<pad>', '<pad>']]



We lower-cased all tokens above; give some reasons why this is a good idea, and why it may be harmful to our embeddings.

[2 marks]

**Good**

Does not differentiate between differentiate between different capitalization between words. Example: CBOW, Cbow, cbow

**Bad**

Incorrectly clumps together it and IT, for example

## Word Embeddings Model

We will implement the CBOW model for constructing word embedding models.

In the CBOW model we try to predict the center word based on the context. That is, we take as input ```n``` context words, encode them as vectors, then combine them by summation. This will give us one embedding. We then use this embedding to predict *which* word in our vocabuary is the most likely center word. 

Implement this model 

[7 marks]

In [47]:
a = torch.randn(2,3,2)
display(a)
b = torch.sum(a,dim= 1)

tensor([[[ 1.1961, -0.6697],
         [ 0.3590,  2.0615],
         [ 0.0976, -0.2442]],

        [[-0.7626, -0.0066],
         [-0.1489, -2.4616],
         [ 0.8705,  1.5389]]])

In [48]:
b.shape

torch.Size([2, 2])

In [ ]:
class CBOWModel(nn.Module):
    def __init__(self, vocab_size, embedding_dim, padding_idx=None):
        super().__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=padding_idx)
        self.prediction = nn.Linear(embedding_dim, vocab_size) # we dont need context_size*embedding_dim because we will apply summation before.
    
    def forward(self, context):
        # context (B, S)
        embedded_context = self.embeddings(context) #(B, S, D)
        projection = self.projection_function(embedded_context)
        predictions = self.prediction(projection)
        #return predictions
        output = F.softmax(predictions, dim=0) # do we need this?
        return output
        
    def projection_function(self, xs):
        #This is alternative to another linear layer. Basically linear layer with same coefficient.
        """
        This function will take as input a tensor of size (B, S, D)
        where B is the batch_size, S the window size, and D the dimensionality of embeddings
        this function should compute the sum over the embedding dimensions of the input, 
        that is, we transform (B, S, D) to (B, 1, D) or (B, D) 
        """
        xs_sum = torch.sum(xs, dim=1) 
        return xs_sum

In [132]:
len(dataset)

1154276

In [136]:

vocab = dataset.word_to_idx
len_vocab = len(vocab)
#dataset, vocab = get_data(...)

# build model and construct loss/optimizer
cbow_model = CBOWModel(len_vocab, word_embeddings_hyperparameters['embedding_dim'], padding_idx=dataset.pad_idx)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(cbow_model.parameters(), lr=word_embeddings_hyperparameters['learning_rate'])

# start training loop

for epoch in range(word_embeddings_hyperparameters['epochs']):
    loader =  get_loader(dataset, word_embeddings_hyperparameters['batch_size'])
    total_loss = 0
    for i, batch in enumerate(loader):
        context = batch.contexts
        #target_word = batch.target_word
        centers = batch.centers
        #Note: this is not nessecary - the loss function understands.
        #targets = torch.zeros(len(centers), len_vocab)
        #for j, idx in enumerate(centers): #one hot encoding
         #   targets[j][idx] = 1
        
        
        # send your batch of sentences to the model
        output = cbow_model(context)
        
        loss = loss_fn(output, centers)
        total_loss += loss.item()
        print(f'Batch {i} : Average Loss = {round(total_loss/(i+1),5)}')#, end='\r')
        
        # calculate gradients
        loss.backward()
        # update model weights
        optimizer.step()
        # reset gradients
        optimizer.zero_grad()

    print(f'Epoch {epoch} : Average Loss = {round(total_loss/(i+1),5)}')#, end='\r')

Batch 0 : Average Loss = 11.22902


In [134]:
# you can change these numbers to suit your needs :)
word_embeddings_hyperparameters = {'epochs':3,
                                   'batch_size':16,
                                   'embedding_size':128, #What is this?
                                   'learning_rate':0.001,
                                   'embedding_dim':128}

word_embeddings_hyperparameters = {'epochs':5,
                                   'batch_size':4096,
                                   'learning_rate':0.005,
                                   'embedding_dim':128}

Train your model. Iterate over the dataset, get outputs from your model, calculate loss and backpropagate.

We mentioned in the lecture that we use Negative Log Likelihood (https://pytorch.org/docs/stable/generated/torch.nn.NLLLoss.html) loss to train Word2Vec model. In this lab we'll take a shortcut when *training* and use Cross Entropy Loss (https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html), basically it combines ```log_softmax``` and ```NLLLoss```. So what your model should output is a *score* for each word in our vocabulary. The ```CrossEntropyLoss``` will then assign probabilities and calculate the negative log likelihood loss.

[3 marks]

In [ ]:
# load data
loader =  get_loader(dataset, word_embeddings_hyperparameters['batch_size'])
vocab = dataset.word_to_idx
len_vocab = len(vocab)
#dataset, vocab = get_data(...)

# build model and construct loss/optimizer
cbow_model = CBOWModel(len_vocab, word_embeddings_hyperparameters['embedding_dim'])
cbow_model.to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(cbow_model.parameters(), lr=word_embeddings_hyperparameters['learning_rate'])

# start training loop
total_loss = 0
for epoch in range(word_embeddings_hyperparameters['epochs']):
    for i, batch in enumerate(loader):
        
        context = batch.contexts
        #target_word = batch.target_word
        centers = batch.centers
        targets = torch.zeros(len(targets), len_vocab)
        for j, idx in enumerate(centers): #one hot encoding
            targets[j][idx] = 1
        
        
        
        # send your batch of sentences to the model
        output = cbow_model(context)
        
        # compute the loss, you'll need to reshape the input
        # you can read more about this is the documentation for
        # CrossEntropyLoss
        loss = loss_fn(output, targets)
        total_loss += loss.item()
        
        # print average loss for the epoch
        
        
        # compute gradients
        #...
        
        # update parameters
        #...
        
        # reset gradients
        #...
    print(f'Epoch {epoch} : Average Loss = {round(total_loss/(i+1),5)}', end='\r') 
    break
    print()
        

In [ ]:
# load data
dataset, vocab = get_data(...)

# build model and construct loss/optimizer
cbow_model = CBOWModel(len(vocab), word_embeddings_hyperparameters['embedding_dim'])
cbow_model.to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(cbow_model.parameters(), lr=word_embeddings_hyperparameters['lr'])

# start training loop
total_loss = 0
for epoch in range(word_embeddings_hyperparameters['epochs']):
    for i, batch in enumerate(dataset):
        
        context = batch.context
        target_word = batch.target_word
        
        # send your batch of sentences to the model
        output = cbow_model(context)
        
        # compute the loss, you'll need to reshape the input
        # you can read more about this is the documentation for
        # CrossEntropyLoss
        loss = loss_fn(...)
        total_loss += loss.item()
        
        # print average loss for the epoch
        print(total_loss/(i+1), end='\r') 
        
        # compute gradients
        ...
        
        # update parameters
        ...
        
        # reset gradients
        ...
    print()
        

## Evaluating the model

We will evaluate the model on a dataset of word similarities, WordSim353 (http://alfonseca.org/eng/research/wordsim353.html , also avalable in vanvas under files/03-l). The first thing we need to do is read the dataset and translate it to integers. What we'll do is to reuse the ```Field``` that records word indexes (the second output of ```get_data()```) and use it to parse the file.

The wordsim data is structured as follows:

```
word1 word2 score
...
```


The ```Field``` we got from ```read_data()``` has two built-in functions, ```stoi``` which maps a string to an integer and ```itos``` which maps an integer to a string. 

What our datareader needs to do is: 

```
for line in file:
    word1, word2, score = file.split()
    # encode word1 and word2 as integers
    word1_idx = vocab.vocab.stoi[word1]
    word2_idx = vocab.vocab.stoi[word2]
```

when we have the integers for ```word_1``` and ```word2``` we'll compute the similarity between their word embeddings with *cosine simlarity*. We can obtain the embeddings by querying the embedding layer of the model.

We calculate the cosine similarity for each word pair in the dataset, then compute the pearson correlation between the similarities we obtained with the scores given in the dataset. 

[4 marks]

In [ ]:
# your code goes here

def read_wordsim(path, vocab, embeddings):
    dataset_sims = []
    model_sims = []
    with open(path) as f:
        for line in f:
            word1, word2, score = f.split()
            
            score = float(score)
            dataset_sims.append(score)
            
            # get the index for the word
            word1_idx = ...
            word2_idx = ...
            
            # get the embedding of the word
            word1_emb = ...
            word2_emb = ...
            
            # compute cosine similarity, we'll use the version included in pytorch functional
            # https://pytorch.org/docs/master/generated/torch.nn.functional.cosine_similarity.html
            cosine_similarity = F.cosine_similarity(...)
            
            model_sims.append(cosine_similarity.item())
    
    return dataset_sims, model_sims

path = 'wordsim_similarity_goldstandard.txt'
data, model = read_wordsim(...)
pearson_correlation = np.corrcoef(data, model)
            
# the non-diagonals give the pearson correlation,
print(pearson_correlation)

Do you think the model performs good or bad? Why?

[3 marks]

Select the 10 best and 10 worst performing word pairs, can you see any patterns that explain why *these* are the best and worst word pairs?

[3 marks]

Suggest some ways of improving the model we apply to WordSim353.

[3 marks]

If we consider a scenario where we use these embeddings in a downstream task, for example sentiment analysis (roughly: determining whether a sentence is positive or negative). 

Give some examples why the sentiment analysis model would benefit from our embeddnings and one examples why our embeddings could hur the performance of the sentiment model.

[3 marks]

# Language modeling

In this second part we'll build a simple LSTM language model. Your task is to construct a model which takes a sentence as input and predict the next word for each word in the sentence. For this you'll use the ```LSTM``` class provided by PyTorch (https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html). You can read more about the LSTM here: https://colah.github.io/posts/2015-08-Understanding-LSTMs/

NOTE!!!: Use the same dataset (wiki-cropus.txt) as before.

Our setup is similar to before, we first encode the words as distributed representations then pass these to the LSTM and for each output we predict the next word.

For this we'll build a new dataloader with torchtext, the file we pass to the dataloader should contain one sentence per line, with words separated by whitespace.

```
word_1, ..., word_n
word_1, ..., word_k
...
```

in this dataloader you want to make sure that each sentence begins with a ```<start>``` token and ends with a ```<end>``` token, there is a keyword argument in ```Field``` for this :). But other than that, as before you read the dataset and output a iterator over the dataset and a vocabulary. 

Implement the dataloader, language model and the training loop (the training loop will basically be the same as for word2vec).

[12 marks]

In [ ]:
# you can change these numbers to suit your needs as before :)
lm_hyperparameters = {'epochs':3,
                      'batch_size':16,
                      'learning_rate':0.001,
                      'embedding_dim':128,
                      'output_dim':128}

In [ ]:
data_path = 'wiki-corpus.txt'
def get_data():
    # your code here, roughly the same as for the word2vec dataloader

In [ ]:
class LM_withLSTM(nn.Module):
    def __init__(...):
        super(LM_withLSTM, self).__init__()
        self.embeddings = ...
        self.LSTM = ...
        self.predict_word = ...
    
    def forward(self, seq):
        embedded_seq = ...
        timestep_reprentation, *_ = ...
        predicted_words = ...
        
        return predicted_words

In [ ]:
# load data
dataset, vocab = get_data(...)

# build model and construct loss/optimizer
lm_model = LM_withLSTM(len(vocab), 
                       lm_hyperparameters['embedding_dim'],
                       lm_hyperparameters['output_dim'])
lm_model.to(device)

loss_fn = CrossEntropyLoss()
optimizer = optim.Adam(cbow_model.parameters(), lr=lm_hyperparameters['lr'])

# start training loop
total_loss = 0
for epoch in range(lm_hyperparameters['epochs']):
    for i, batch in enumerate(dataset):
        
        # the strucure for each BATCH is:
        # <start>, w0, ..., wn, <end>
        sentence = batch.sentence
        
        # when training the model, at each input we predict the *NEXT* token
        # consequently there is nothing to predict when we give the model 
        # <end> as input. 
        # thus, we do not want to give <end> as input to the model, select 
        # from each batch all tokens except the last. 
        # tip: use pytorch indexing/slicing (same as numpy) 
        # (https://pytorch.org/tutorials/beginner/basics/tensorqs_tutorial.html#operations-on-tensors)
        # (https://jhui.github.io/2018/02/09/PyTorch-Basic-operations/)
        input_sentence = ...
        
        # send your batch of sentences to the model
        output = lm_model(input_sentence)
        
        # for each output, the model predict the NEXT token, so we have to reshape 
        # our dataset again. On timestep t, we evaluate on token t+1. That is,
        # we never predict the <start> token ;) so this time, we select all but the first 
        # token from sentences (that is, all the tokens that we predict)
        gold_data = ...
        
        # the shape of the output and sentence variable need to be changed,
        # for the loss function. Details are in the documentation.
        # You can use .view(...,...) to reshape the tensors  
        loss = loss_fn(...)
        total_loss += loss.item()
        
        # print average loss for the epoch
        print(total_loss/(i+1), end='\r') 
        
        # compute gradients
        ...
        
        # update parameters
        ...
        
        # reset gradients
        ...
    print()

### Evaluating the language model

We'll evaluate our model using the BLiMP dataset (https://github.com/alexwarstadt/blimp). The BLiMP dataset contains sets of linguistic minimal pairs for various syntactic and semantic phenomena, We'll evaluate our model on *existential quantifiers* (link: https://github.com/alexwarstadt/blimp/blob/master/data/existential_there_quantifiers_1.jsonl). This data, as the name suggests, investigate whether language models assign higher probability to *correct* usage of there-quantifiers. 

An example entry in the dataset is: 

```
{"sentence_good": "There was a documentary about music irritating Allison.", "sentence_bad": "There was each documentary about music irritating Allison.", "field": "semantics", "linguistics_term": "quantifiers", "UID": "existential_there_quantifiers_1", "simple_LM_method": true, "one_prefix_method": false, "two_prefix_method": false, "lexically_identical": false, "pairID": "0"}
```

Download the dataset and build a datareader (similar to what you did for word2vec). The dataset structure you should aim for is (you don't need to worry about the other keys for this assignment):

```
good_sentence_1, bad_sentence_1
...
```

your task now is to compare the probability assigned to the good sentence with to the probability assigned to the bad sentence. To compute a probability for a sentence we consider the product of the probabilities assigned to the *gold* tokens, remember, at timestep ```t``` we're predicting which token comes *next* e.g. ```t+1``` (basically, you do the same thing as you did when training).

In rough pseudo code what your code should do is:

```
accuracy = []
for good_sentence, bad_sentence in dataset:
    gs_lm_output = LanguageModel(good_sentence)
    gs_token_probabilities = softmax(gs_lm_output)
    gs_sentence_probability = product(gs_token_probabilities[GOLD_TOKENS])

    bs_lm_output = LanguageModel(bad_sentence)
    bs_token_probabilities = softmax(bs_lm_output)
    bs_sentence_probability = product(bs_token_probabilities[GOLD_TOKENS])

    # int(True) = 1 and int(False) = 0
    is_correct = int(gs_sentence_probability > bs_sentence_probability)
    accuracy.append(is_correct)

print(numpy.mean(accuracy))
    
```

[6 marks]

In [ ]:
# your code goes here
import json

def evaluate_model(path, vocab, model):
    
    accuracy = []
    with open(path) as f:
        # iterate over one pair of sentences at a time
        for line in f:
            # load the data
            data = json.loads(line)
            good_s = data['sentence_good']
            bad_s = data['sentence_bad']
            
            # the data is tokenized as whitespace
            tok_good_s = ...
            tok_bad_s = ...
            
            # encode your words as integers using the vocab from the dataloader, size is (S)
            # we use unsqueeze to create the batch dimension 
            # in this case our input is only ONE batch, so the size of the tensor becomes: 
            # (S) -> (1, S) as the model expects batches
            enc_good_s = torch.tensor([_ for x in tok_good_s], device=device).unsqueeze(0)
            enc_bad_s = torch.tensor([_ for x in tok_bad_s], device=device).unsqueeze(0)
            
            # pass your encoded sentences to the model and predict the next tokens
            good_s = LM_withLSTM(enc_good_s)
            bad_s = LM_withLSTM(enc_bad_s)
            
            # get probabilities with softmax
            gs_probs = F.softmax(...)
            bs_probs = F.softmax(...)
            
            # select the probability of the gold tokens
            gs_sent_prob = find_token_probs(gs_probs, enc_good_s)
            bs_sent_prob = find_token_probs(bs_probs, enc_bad_s)
            
            accuracy.append(int(gs_sent_prob>bs_sent_prob))
            
    return accuracy
            
def find_token_probs(model_probs, encoded_sentece):
    probs = []

    # iterate over the tokens in your encoded sentence
    for token, gold_token in enumerate(encoded_sentece):
        # select the probability of the gold tokens and save
        # hint: pytorch indexing is helpful here ;)
        prob = ...
        probs.append(prob)
    sentence_prob = ...
    return sentence_prob

path = 'existential_there_quantifiers_1.jsonl'
accuracy = evaluate_model(path, ..., ...)

print('Final accuracy:')
print(np.round(np.mean(accuracy), 3))


### Analysis

Our model get some score, say, 55% correct predictions. Is this good? Suggest some *baseline* (i.e. a stupid "model" we hope ours is better than) we can compare the model against.

[3 marks]

Suggest some improvements you could make to your language model.

[3 marks]

Suggest some other metrics we can use to evaluate our system

[2 marks]

# Literature


Neural architectures:
* Y. Bengio, R. Ducharme, P. Vincent, and C. Janvin. A neural probabilistic language model. (Links to an external site.) Journal of Machine Learning Research, 3(6):1137–1155, 2003. (Sections 3 and 4 are less relevant today and hence you can glance through them quickly. Instead, look at the Mikolov papers where they describe training word embeddings with the current neural network architectures.)
* T. Mikolov, K. Chen, G. Corrado, and J. Dean. Efficient estimation of word representations in vector space. arXiv preprint arXiv:1301.3781, 2013.
* T. Mikolov, I. Sutskever, K. Chen, G. S. Corrado, and J. Dean. Distributed representations of words and phrases and their compositionality. In Advances in neural information processing systems, pages 3111–3119, 2013.
    


Total marks: 63